In [ ]:
import pandas as pd

# Läser in fliken "Veckodata Riket" från datafilen "fhm_covid19".
df_covid19 = pd.read_excel('Data/fhm_covid19.xlsx', sheet_name='Veckodata Riket')

# En initial dataanalys av "Veckodata Riket".
print(df_covid19.info())
print(df_covid19.describe())
print(df_covid19['Antal_fall_100000inv_14dagar'].value_counts())
print(df_covid19.head())
print(df_covid19.columns)
print(df_covid19.index)

In [ ]:
# Konkatenera kolumnerna "år" och "veckonummer" till en "Vecka"-kolumn med formatet 'ÅrVeckonummer'.
# Formaterar veckonumret så att det alltid har två siffror genom att lägga till en inledande nolla vid ental. 
df_covid19['Vecka'] = df_covid19['år'].astype(str) + "v" + df_covid19['veckonummer'].astype(str).str.zfill(2) 

print(df_covid19['Vecka'].head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Förbereder subplottens storlek.
fig, axs = plt.subplots(2, 2, figsize=(14, 10)) # Justerad storlek för bättre visuell representation.

# Anpassar antalet etiketter som visas på x-axeln till var fjärde vecka för att undvika överlappning.
# Skapar en variabel för att justera x-axel etiketter för läsbarhet.
ticks_to_show = df_covid19['Vecka'].iloc[::4]
set_rotation = 90

# Första subplot-linjediagrammet över antalet avlidna fall per vecka.
sns.lineplot(data=df_covid19, x='Vecka', y='Antal_avlidna_vecka', ax=axs[0, 0])
axs[0, 0].set_title("Antal avlidna fall per vecka")
axs[0, 0].set_xticks(ticks_to_show.index)
axs[0, 0].set_xticklabels(ticks_to_show.values, rotation= set_rotation)

# Andra subplot-linjediagrammet över antalet fall per vecka.
sns.lineplot(data=df_covid19, x='Vecka', y='Antal_fall_vecka', ax=axs[0, 1])
axs[0, 1].set_title("Nya fall per vecka")
axs[0, 1].set_xticks(ticks_to_show.index)
axs[0, 1].set_xticklabels(ticks_to_show.values, rotation= set_rotation)

# Tredje subplot-linjediagrammet är ett kombinerat linjediagram för att visa trendea av avlidna och nya COVID-19 fall per vecka.
# Smälter data för att få 'Antal_avlidna_vecka' och 'Antal_fall_vecka' i samma kolumn.
melted_df = df_covid19.melt(id_vars=['Vecka'], value_vars=['Antal_avlidna_vecka', 'Antal_fall_vecka'], var_name='Kategori', value_name='Antal')

# Använder 'hue' parametern som automatiskt både tilldelar olika färger till varje kategori
# och skapar en legend för att förklara vilken färg som motsvarar vilken kategori.
sns.lineplot(x='Vecka', y='Antal', hue='Kategori', data=melted_df, ax=axs[1, 0])
axs[1, 0].set_title("Kombinerade avlidna och nya fall per vecka")
axs[1, 0].set_xticks(ticks_to_show.index)
axs[1, 0].set_xticklabels(ticks_to_show.values, rotation= set_rotation)

# Fjärde subplot-linjediagrammet som är för kumulativa antal fall.
sns.lineplot(data=df_covid19, x='Vecka', y='Kum_antal_fall', ax=axs[1, 1])
axs[1, 1].set_title("Kumulativa antal fall per vecka")
axs[1, 1].set_xticks(ticks_to_show.index)
axs[1, 1].set_xticklabels(ticks_to_show.values, rotation= set_rotation)

# Anpassar layouten så att allt får plats och inget klipps bort i den sparade PNG-filen.
plt.tight_layout()

# Sparar diagrammet som en PNG-fil med en vit bakgrundsfärg för att undvika att den visas som transparent.
plt.savefig('Visualisering/2x2_covid19_linjediagram.png', facecolor='w')

# Visar plottarna
plt.show()

In [ ]:
import plotly.express as px

# Skapar ett linjediagram för antalet avlidna fall per vecka.
fig = px.line(df_covid19, x='Vecka', y='Antal_avlidna_vecka', title='Antal avlidna fall per vecka')

# Förbättrar layouten.
fig.update_layout(
    xaxis_title='Vecka',
    yaxis_title='Antal avlidna',
    xaxis=dict(
        tickmode='array',
        tickvals=df_covid19['Vecka'][::2]), # Anpassar antalet etiketter som visas på x-axeln till varannan vecka för att undvika överlappning .

    xaxis_tickangle= -90  # Roterar etiketterna på x-axeln för bättre läsbarhet.
)

fig.show()
# Sparar grafen som en HTML-fil.
fig.write_html('Visualisering/avlidna_per_vecka.html')

# Linjediagram för nya fall per vecka.
fig = px.line(df_covid19, x='Vecka', y='Antal_fall_vecka', title='Nya fall per vecka')
fig.update_layout(xaxis_title='Vecka', yaxis_title='Antal nya fall', xaxis=dict(tickmode='array', tickvals=df_covid19['Vecka'][::2]), xaxis_tickangle=-90)
fig.show()
fig.write_html('Visualisering/nya_fall_per_vecka.html')

# Linjediagram för avlidna och nya fall per vecka i samma fönster.
# 'color' parametern tilldelar olika färger baserat på värdena i 'Kategori' kolumnen.
fig = px.line(melted_df, x='Vecka', y='Antal', color='Kategori', title='Kombinerade avlidna och nya fall per vecka')
fig.update_layout(xaxis_title='Vecka', yaxis_title='Antal fall', xaxis=dict(tickmode='array', tickvals=melted_df['Vecka'][::2]), xaxis_tickangle=-90)
fig.show()
fig.write_html('Visualisering/kombinerade_avlidna_och_nya_fall_per_vecka.html')

# Linjediagram för kumulativt antal fall per vecka.
fig = px.line(df_covid19, x='Vecka', y='Kum_antal_fall', title='Kumulativa antal fall per vecka')
fig.update_layout(xaxis_title='Vecka', yaxis_title='Kumulativt antal fall', xaxis=dict(tickmode='array', tickvals=df_covid19['Vecka'][::2]), xaxis_tickangle=-90)
fig.show()
fig.write_html('Visualisering/kumulativa_fall_per_vecka.html')

In [7]:
# Läser in fliken "Vaccinerade kommun och ålder" från datafilen "fhm_covid19_vaccin".
df_covid19_vaccin = pd.read_excel('Data/fhm_covid19_vaccin.xlsx', sheet_name='Vaccinerade kommun och ålder')

# Räknar antalet län
antal_lan = df_covid19_vaccin['Län'].nunique()
print(f"Antalet län som är representerade i datasetet: {antal_lan}")

# Räknar antelet kommuner
antal_kommuner = df_covid19_vaccin['Kommun'].nunique()
print(f"Antalet kommuner som är representerade i datasetet: {antal_kommuner}")

# Summa av befolkningen
total_befolkning = df_covid19_vaccin['Befolkning'].sum()
print(f"Totala befolkningen som är representerad i datasetet: {total_befolkning}")

Antalet län som är representerade i datasetet: 21
Antalet kommuner som är representerade i datasetet: 290
Totala befolkningen som är representerad i datasetet: 9092790
